In [14]:
import pymysql
conn = pymysql.connect(host = '192.168.1.237',
                      user = 'root',
                      port = 3306,
                      password = 'acorn1234Q!',
                      db = 'acornschool',
                      charset = 'utf8mb4',
                      use_unicode = True,
                      cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()
cur.execute("""CREATE TABLE pet(
                    id INT NOT NULL AUTO_INCREMENT, 
                    name VARCHAR(20), 
                    owner VARCHAR(20), 
                    species VARCHAR(20), 
                    sex CHAR(1), 
                    birth DATE, 
                    death DATE, 
                    PRIMARY KEY (id)) ENGINE=InnoDB AUTO_INCREMENT=1""" )
print(cur.description)

# 네트워크 접속은 OS에서 관리한다. connection은 open하면 반드시 close해야한다.
conn.close()

None


In [27]:
conn = pymysql.connect(host = '192.168.1.237',
                      user = 'root',
                      port = 3306,
                      password = 'acorn1234Q!',
                      db = 'acornschool',
                      charset = 'utf8mb4',
                      use_unicode = True,
                      cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()

# execute : 1건
sql = """INSERT INTO pet(name, owner, species, sex, birth, death)
              VALUES (%s, %s, %s, %s, %s, %s)"""

cur.execute(sql, ("유리", "acorn", "고양이", "f", "20140101", "20160201"))
cur.execute(sql, ("귀염", "acorn", "강아지", "f", "20140301", "20160901"))
cur.execute(sql, ("돌이", "acorn", "곰", "f", "20140801", "20180201"))
conn.commit()

# executemany : 다건
# DB:NULL = Python:None
lis = [("대한", "민주", "cat", "f", "20010204", None),
       ("민국", "자유", "cat", "f", "20100317", None),
       ("만세", "민주", "cat", "f", "20100513", None),
       ("하자", "민주", "cat", "m", "20150827", None),
       ("성공", "자유", "cat", "m", "20170831", "20180429")]
cur.executemany(sql, lis)
conn.commit()

print(cur.rowcount)

conn.close()

5


In [29]:
conn = pymysql.connect(host = '192.168.1.237',
                      user = 'root',
                      port = 3306,
                      password = 'acorn1234Q!',
                      db = 'acornschool',
                      charset = 'utf8mb4',
                      use_unicode = True,
                      cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()

cur.execute("SELECT * FROM pet") # cursor recordSet 생성
print(cur.rowcount)
cur.description

cur.fetchone()
cur.fetchone()
cur.fetchone()
recs = cur.fetchmany(2)
for rec in recs:
    print(rec)
print()

recs = cur.fetchall()
for rec in recs:
    print(rec)
    
cur.close()
conn.close()

8
{'id': 4, 'name': '대한', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2001, 2, 4), 'death': None}
{'id': 5, 'name': '민국', 'owner': '자유', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 3, 17), 'death': None}

{'id': 6, 'name': '만세', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 5, 13), 'death': None}
{'id': 7, 'name': '하자', 'owner': '민주', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2015, 8, 27), 'death': None}
{'id': 8, 'name': '성공', 'owner': '자유', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2017, 8, 31), 'death': datetime.date(2018, 4, 29)}


In [41]:
# 문제. 생일이 2015-01-01이후인 pet을 출력하시요.
conn = pymysql.connect(host = '192.168.1.237',
                      user = 'root',
                      port = 3306,
                      password = 'acorn1234Q!',
                      db = 'acornschool',
                      charset = 'utf8mb4',
                      use_unicode = True,
                      cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()

sql = "SELECT * FROM pet WHERE birth > 20150101"
cur.execute(sql)

recs = cur.fetchall()
for rec in recs:
    val = list(rec.values())
    print("ID=%d, Name=%s, 주인=%s" % (val[0], val[1], val[2]))
    
# "유리"->"초심"으로 변경
sql = "UPDATE pet SET name = %s WHERE name = %s"
cur.execute(sql, ("초심", "유리"))
conn.commit()
print(cur.rowcount)

# "초심" 삭제
sql = "DELETE FROM pet WHERE name = %s"
cur.execute(sql, "초심")
conn.commit()
print(cur.rowcount)

cur.close()
conn.close()

ID=7, Name=하자, 주인=민주
ID=8, Name=성공, 주인=자유
0
0


In [43]:
conn = pymysql.connect(host = '192.168.1.237',
                      user = 'root',
                      port = 3306,
                      password = 'acorn1234Q!',
                      db = 'acornschool',
                      charset = 'utf8mb4',
                      use_unicode = True,
                      cursorclass = pymysql.cursors.DictCursor)

cur = conn.cursor()

cur.callproc("pet_select")
if( cur.rowcount ):
    print(cur.fetchall())
    
cur.close()
conn.close()

[{'id': 2, 'name': '귀염', 'owner': 'acorn', 'species': '강아지', 'sex': 'f', 'birth': datetime.date(2014, 3, 1), 'death': datetime.date(2016, 9, 1)}, {'id': 3, 'name': '돌이', 'owner': 'acorn', 'species': '곰', 'sex': 'f', 'birth': datetime.date(2014, 8, 1), 'death': datetime.date(2018, 2, 1)}, {'id': 4, 'name': '대한', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2001, 2, 4), 'death': None}, {'id': 5, 'name': '민국', 'owner': '자유', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 3, 17), 'death': None}, {'id': 6, 'name': '만세', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 5, 13), 'death': None}, {'id': 7, 'name': '하자', 'owner': '민주', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2015, 8, 27), 'death': None}, {'id': 8, 'name': '성공', 'owner': '자유', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2017, 8, 31), 'death': datetime.date(2018, 4, 29)}]


In [47]:
# insert procedure 이용
# '마음이', 'acorn', '백구', 'f', '20180102', '20181104'
conn = pymysql.connect(host = '192.168.1.237',
                      user = 'root',
                      port = 3306,
                      password = 'acorn1234Q!',
                      db = 'acornschool',
                      charset = 'utf8mb4',
                      use_unicode = True,
                      cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()

data = ('마음이', 'acorn', '백구', 'f', '20180102', '20181104')
cur.callproc("pet_insert", data)
conn.commit()

cur.close()
conn.close()

In [59]:
# 문제. 이름을 update하는 프로시져를 작성하고 '마음이'를 '마음'으로 수정하시요.
conn = pymysql.connect(host = '192.168.1.237',
                      user = 'root',
                      port = 3306,
                      password = 'acorn1234Q!',
                      db = 'acornschool',
                      charset = 'utf8mb4',
                      use_unicode = True,
                      cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()

data = ("마음이", "마음", 0)
cur.callproc("pet_update", data)
cur.execute("SELECT @_pet_update_2") # procedure이름_2에 결과값(res)이 저장됨

result = cur.fetchone() # result = key:data 형식
print(result['@_pet_update_2'])

if result['@_pet_update_2'] == 2:
    print("수정할 이름이 없습니다.")
else:
    print("성공적으로 수정되었습니다.")

cur.close()
conn.close()

2
수정할 이름이 없습니다.


In [61]:
conn = pymysql.connect(host = '192.168.1.237',
                      user = 'root',
                      port = 3306,
                      password = 'acorn1234Q!',
                      db = 'acornschool',
                      charset = 'utf8mb4',
                      use_unicode = True,
                      cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()

data = ("귀염", 0)
cur.callproc("pet_delete", data)
conn.commit()

cur.execute("SELECT @_pet_delete_1")
result = cur.fetchone()
print(result["@_pet_delete_1"])
if result["@_pet_delete_1"] == 2 :
    print("삭제할 이름이 없습니다.")
else:
    print("성공적으로 삭제하였습니다.")

cur.close()
conn.close()

2
삭제할 이름이 없습니다.


In [68]:
conn = pymysql.connect(host = '192.168.1.237',
                      user = 'root',
                      port = 3306,
                      password = 'acorn1234Q!',
                      db = 'acornschool',
                      charset = 'utf8mb4',
                      use_unicode = True,
                      cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()

# Exception 예외처리 : 프로그램은 지속적으로 실행되어야 한다.
try:
    data = ("귀염", 0)
    cur.callproc("pet_delete", data)
    conn.commit()
    
    cur.execute("SELECT @_pet_delete_1")
    result = cur.fetchone()
    print(result["@_pet_delete_1"])
    
    if result["@_pet_delete_1"] == 2 :
        print("삭제할 이름이 없습니다.")
    else:
        print("성공적으로 삭제하였습니다.")
        
except:
    print("Exception!!!")
    conn.rollback()

cur.close();
conn.close();

2
삭제할 이름이 없습니다.


In [8]:
class Student:
    def __init__(self):
        self.bunho = 0
        self.name = ""
        self.kor = 0
        self.eng = 0
        self.mat = 0
        self.total = 0
        self.average = 0
        self.grade = ""
        
    def inputData(self):
        self.name = input("이름을 입력하세요 >>")
        self.kor = eval(input("국어 점수 >>"))
        self.mat = eval(input("수학 점수 >>"))
        self.eng = eval(input("영어 점수 >>"))
        
    def calc_total_average(self):
        self.total = self.kor + self.mat + self.eng
        self.average = round(self.total / 3, 2)
        self.grade = self.calcSemGrade()
        
    def calcSemGrade(self):
        if self.average >= 90:
            return "A"
        elif self.average >= 80:
            return "B"
        elif self.average >= 70:
            return "C"
        elif self.average >= 60:
            return "D"
        else:
            return "F"
        
    def __str__(self):
        return "%5s %5s %6s %6s %6s %7s %7s %s" % (self.bunho, self.name, self.kor, self.mat, self.eng,
                self.total, self.average, self.grade)
        
    def __cmp__(self, other):
        return self.name == other.name
    
    def __call__(self, other): # class를 함수처럼 사용하고자 할때 overriding
        st = []
        
        for student in other:
            s = Student()
            s.bunho = student[0]
            s.name = student[1]
            s.kor = student[2]
            s.eng = student[3]
            s.mat = student[4]
            
            st.append(s)
            
        return st

In [9]:
import pymysql
class maria:
    schoolname = "에이콘 아카데미"
    
    def __init__(self):
        self.conn = pymysql.connect(host='192.168.1.237', port=3306, user='root',
                           password='acorn1234Q!',db='acornschool', charset='utf8',
                           autocommit=True)
        cor = self.conn.cursor()
        cor.execute("SHOW TABLES")
        tables = cor.fetchall()
        c = []
        for i in tables:
            if  i == ('sungjuk',):
                c.append(i)
        if len(c) != 0:
            sungjuk_table = "SELECT * FROM sungjuk"
            cor.execute(sungjuk_table)
            print('테이블이 있습니다. ')
        else:
            cor.execute("CREATE TABLE sungjuk (name VARCHAR(20), kor INTEGER(20), mat INTEGER(20), eng INTEGER(20))")
            print('sungjuk테이블을 생성하였습니다.')
    def __del__(self):
        self.conn.close()

    def selectdb(self):
        cur = self.conn.cursor()
        cur.callproc("sungjuk_select")
        if (cur.rowcount):
            stu = list(cur.fetchall())
        else :
            stu = 0;
        cur.close()
        return stu
   
    def insertdb(self, name, kor, mat, eng):
        cur = self.conn.cursor()
        args = (name, kor, mat, eng)
        cur.callproc("sungjuk_insert",args)
        cur.close()
       
    def updatedb(self, in_name, up_name):
        cur = self.conn.cursor()
        args = (in_name, up_name, 0)
        cur.callproc("sungjuk_update", args)
        cur.execute('SELECT @_sungjuk_update_2')
        result = cur.fetchone()
        if result == 2 :
            print("수정할 이름이 없습니다.")
        else :
            print("성공적으로 수정하였습니다.")
        cur.close()
        return result 
   
    def deletedb(self,in_name):
        cur = self.conn.cursor()
        args = (in_name, 0)
        cur.callproc("sungjuk_delete", args)
        cur.execute('SELECT @_sungjuk_delete_1')
        result = cur.fetchone()
        if result == 2 :
            print("삭제할 이름이 없습니다.")
        else :
            print("성공적으로 삭제하였습니다.")
        cur.close()
        return result

In [10]:
from math import  *
from statistics import *

class Management:
    def __init__(self):
        self.maria = maria()
        
    def M_select(self):
        data = self.maria.selectdb()
        st = []
        if (data):
            for student in data:
                stp = Student()
                stp.bunho = student[0]
                stp.name = student[1]
                stp.kor = student[2]
                stp.eng = student[3]
                stp.mat = student[4]
                
                st.append(stp)                    
            # studdata=Student((data))
            print(maria.schoolname + "  성적 계산표 ")
            print('%5s %5s %6s %6s %6s %6s  %6s %6s' % ('번 호', '이 름', '국 어',
                    '수 학', '영  어', '총 점', '평 균', '학 점' ))
            
            for stud in st:
                print(stud)                     
        else :
            print("데이터가 없습니다. . ")
           
    def M_insert(self):
        stu = Student()
        stu.inputData()
        self.maria.insertdb(stu.name, stu.kor, stu.mat, stu.eng)
        
    def M_update(self):
        name = input("검색할 이름을 입력하시요")
        replace_name = input("수정할 이름을 입력하시요")
        result = self.maria.updatedb(name, replace_name)
        
    def M_delete(self):       
        name = input("검색할 이름을 입력하시요")
        self.maria.deletedb(name)

In [11]:
import sys
class Menu():
    def __init__(self):
        self.manage = Management()
       
    def work(self):
        menu = [ "전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)"]
        manage = Management()
        while 1:
            sel = input(menu)
            if  sel == "1":
                manage.M_select()
                continue
            elif sel == "2":
                manage.M_insert()
                continue       
            elif sel =="3":
                manage.M_update()
                continue
            elif sel =="4":
                manage.M_delete()
            elif sel =="9":
                del(manage)
                sys.exit()
            else :
                print("잘못된 입력입니다. 다시입력 하시요")
                continue
           
menu = Menu()
menu.work() 

테이블이 있습니다. 
테이블이 있습니다. 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
에이콘 아카데미  성적 계산표 
  번 호   이 름    국 어    수 학   영  어    총 점     평 균    학 점
    2   홍길동     90     80     45       0       0 
    3   김둘리     67     56    100       0       0 
    4   최또치     90     70     60       0       0 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']4
검색할 이름을 입력하시요박또치
성공적으로 삭제하였습니다.
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
에이콘 아카데미  성적 계산표 
  번 호   이 름    국 어    수 학   영  어    총 점     평 균    학 점
    2   홍길동     90     80     45       0       0 
    3   김둘리     67     56    100       0       0 
    4   최또치     90     70     60       0       0 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']4
검색할 이름을 입력하시요박똥치
성공적으로 삭제하였습니다.
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']9


SystemExit: 